In `acl.all.tsv` you'll find 7,188 papers published at major NLP venues (ACL, EMNLP, NAACL, TACL, etc.) between 2013 and 2020.  Your job is to use topic modeling to discover what topics in NLP have been increasing or decreasing in use over this time.  Here is a sample of the data we'll use:

|id|year of publication|title|abstract|
|---|---|---|---|
|pimentel-etal-2020-phonotactic|2020|Phonotactic Complexity and Its Trade-offs|We present methods for calculating a measure of phonotactic complexity---bits per phoneme--- that permits a straightforward cross-linguistic comparison. When given a word, represented as a sequence of phonemic segments such as symbols in the international phonetic alphabet, and a statistical model trained on a sample of word types from the language, we can approximately measure bits per phoneme using the negative log-probability of that word under the model. This simple measure allows us to compare the entropy across languages, giving insight into how complex a language's phonotactics is. Using a collection of 1016 basic concept words across 106 languages, we demonstrate a very strong negative correlation of − 0.74 between bits per phoneme and the average length of words.|
|wang-etal-2020-amr|2020|AMR-To-Text Generation with Graph Transformer|Abstract meaning representation (AMR)-to-text generation is the challenging task of generating natural language texts from AMR graphs, where nodes represent concepts and edges denote relations. The current state-of-the-art methods use graph-to-sequence models; however, they still cannot significantly outperform the previous sequence-to-sequence models or statistical approaches. In this paper, we propose a novel graph-to-sequence model (Graph Transformer) to address this task. The model directly encodes the AMR graphs and learns the node representations. A pairwise interaction function is used for computing the semantic relations between the concepts. Moreover, attention mechanisms are used for aggregating the information from the incoming and outgoing neighbors, which help the model to capture the semantic information effectively. Our model outperforms the state-of-the-art neural approach by 1.5 BLEU points on LDC2015E86 and 4.8 BLEU points on LDC2017T10 and achieves new state-of-the-art performances.|


First, let's read in the data and train a topic model on it (you are free to set the number of topics $K$ as you see fit.

In [ ]:
K=50

In [ ]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [ ]:
def read_docs(dataFile, stopwords):
    
    names=[]    
    docs=[]
   
    with open(dataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            year=int(cols[1])
        
            text=cols[3]
            
            tokens=nltk.word_tokenize(text.lower())
            tokens=[x for x in tokens if filter(x, stopwords)]
            docs.append(tokens)
            names.append((name, year))
    return docs, names

In [ ]:
dataFile="../data/acl.all.tsv"
data, doc_names=read_docs(dataFile, stop_words)

We will convert the data into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [ ]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [ ]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

Now let's run a topic model on this data using gensim's built-in LDA.

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=K, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [ ]:
for i in range(K):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

Now let's print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$ -- in order to ground these topic summaries with actual documents that contain those topics.

In [ ]:
topic_model=lda_model 

topic_docs=[]
for i in range(K):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(K):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

**Q1**: Use this basic framework to plot the distribution of a topic over time (between the years 2013-2020).  Specifically, given a document-topic distribution $\theta$ for an entire corpus such that $\theta_d$ is the document-topic distribution for document $d$ and $\theta_{d,i}$ is the probability of topic $i$ in document $d$, the relative frequency of topic $i$ at time $t$ is the sum of $\theta_{d,i}$ for all documents that are published in year $t$, divided by the total number of documents published in year $t$: 

$$
f(t, i) ={ \sum_{d \in D: date(d) = t} \theta_{d,i} \over \sum_{d \in D: date(d) = t} 1}
$$

For all of the $K$ topics you learn above, plot the distribution of that topic over time (i.e., the x-axis should be time from 2013-2020 and the y-axis should be the relative frequency value $f$ defined above).  Your output here should be $K$ line charts paired with their topic signatures (e.g., from `topic_model.show_topic`, so that we know what topic the chart corresponds to).  See below for sample code generating such a line chart with x, y inputs.

In [ ]:
import matplotlib.pyplot as plt

a=[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
b=[0.4, 0.3, 0.1, 0.9, 0.04, 0.5, 0.45, 0.3 ]

def plot_category(a,b):
    plt.plot(a,b)
    plt.show()

plot_category(a,b)